In [30]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

In [31]:
df=pd.read_csv(r'C:/Users/Acer/Downloads/credit_risk.csv')

In [32]:
df

,age,salary,home_ownership,employment_time,loan_purposes,credit_score,Credit_Amount,loan_rate,Credit_Status,loan_percentage,Payment_History,Credit_History_Length
0,23,70000,RENT,3.0,EDUCATION,A,12000,7.90,0,0.17,N,2
1,25,25000,RENT,0.0,MEDICAL,C,8500,12.53,1,0.34,Y,2
2,24,67000,MORTGAGE,2.0,HOMEIMPROVEMENT,B,12150,10.37,0,0.18,N,3
3,23,69000,RENT,7.0,MEDICAL,A,6000,7.88,0,0.09,N,2
4,30,33000,MORTGAGE,2.0,DEBTCONSOLIDATION,A,6000,8.49,0,0.18,N,8
...,...,...,...,...,...,...,...,...,...,...,...,...
32576,32,58800,OWN,16.0,PERSONAL,C,7000,13.99,0,0.12,N,7
32577,33,30000,RENT,2.0,VENTURE,E,6000,19.42,1,0.20,N,10
32578,39,18300,RENT,0.0,DEBTCONSOLIDATION,D,1200,14.74,1,0.07,Y,11
32579,28,36000,MORTGAGE,NaN,VENTURE,D,6000,15.21,0,0.17,N,7


In [33]:
del df['credit_score']
del df['Payment_History']
del df['Credit_History_Length']

In [34]:
df.isna().sum()

age                   0
salary                0
home_ownership        0
employment_time     895
loan_purposes         0
Credit_Amount         0
loan_rate          3116
Credit_Status         0
loan_percentage       0
dtype: int64

In [35]:
df['salary'].min()

4000

In [36]:
df['salary'].max()-df['salary'].min()

5996000

In [37]:
895/32581

0.027469997851508547

In [38]:
df=df.dropna()

In [39]:
df[df['home_ownership']=='OTHER']

,age,salary,home_ownership,employment_time,loan_purposes,Credit_Amount,loan_rate,Credit_Status,loan_percentage
150,27,63000,OTHER,2.0,DEBTCONSOLIDATION,4500,9.63,0,0.07
194,31,82656,OTHER,10.0,DEBTCONSOLIDATION,6000,12.09,0,0.07
324,36,192000,OTHER,11.0,VENTURE,20000,11.48,0,0.10
365,21,42000,OTHER,5.0,DEBTCONSOLIDATION,12000,14.26,0,0.29
613,24,45000,OTHER,8.0,MEDICAL,7500,8.32,0,0.17
...,...,...,...,...,...,...,...,...,...
31238,28,186000,OTHER,3.0,VENTURE,2500,7.40,0,0.01
31598,24,49000,OTHER,1.0,PERSONAL,20000,11.34,1,0.41
31603,23,73000,OTHER,1.0,PERSONAL,25000,17.90,1,0.34
32420,31,85000,OTHER,4.0,HOMEIMPROVEMENT,16000,11.89,0,0.19


In [40]:
#scaling 
#(value-min)/range
#range=max-min
def scale (col_name):
    df[col_name]=(df[col_name].values-df[col_name].min())/(df[col_name].max()-df[col_name].min())
    return df

In [41]:
scale('age')
scale('salary')

,age,salary,home_ownership,employment_time,loan_purposes,Credit_Amount,loan_rate,Credit_Status,loan_percentage
0,0.024194,0.011007,RENT,3.0,EDUCATION,12000,7.90,0,0.17
1,0.040323,0.003502,RENT,0.0,MEDICAL,8500,12.53,1,0.34
2,0.032258,0.010507,MORTGAGE,2.0,HOMEIMPROVEMENT,12150,10.37,0,0.18
3,0.024194,0.010841,RENT,7.0,MEDICAL,6000,7.88,0,0.09
4,0.080645,0.004837,MORTGAGE,2.0,DEBTCONSOLIDATION,6000,8.49,0,0.18
...,...,...,...,...,...,...,...,...,...
32575,0.048387,0.009540,RENT,3.0,HOMEIMPROVEMENT,7150,6.99,0,0.12
32576,0.096774,0.009139,OWN,16.0,PERSONAL,7000,13.99,0,0.12
32577,0.104839,0.004336,RENT,2.0,VENTURE,6000,19.42,1,0.20
32578,0.153226,0.002385,RENT,0.0,DEBTCONSOLIDATION,1200,14.74,1,0.07


In [42]:
for i in df.select_dtypes(include=['object']):
    print(i, df[i].nunique(), df[i].unique())

home_ownership 4 ['RENT' 'MORTGAGE' 'OWN' 'OTHER']
loan_purposes 6 ['EDUCATION' 'MEDICAL' 'HOMEIMPROVEMENT' 'DEBTCONSOLIDATION' 'VENTURE'
 'PERSONAL']


In [43]:
df['loan_percentage'].nunique()

77

In [44]:
def getTarget(df, column, target):
    vmap = {}
    for c in df[column].unique():
        vmap[c] =  df[ df[column] == c ][ target ].mean()

    df[ column ] = df[column].map( vmap )

    return df

In [45]:
getTarget(df, 'home_ownership', 'Credit_Status' )
getTarget(df, 'loan_purposes', 'Credit_Status' )


,age,salary,home_ownership,employment_time,loan_purposes,Credit_Amount,loan_rate,Credit_Status,loan_percentage
0,0.024194,0.011007,0.312350,3.0,0.170231,12000,7.90,0,0.17
1,0.040323,0.003502,0.312350,0.0,0.268468,8500,12.53,1,0.34
2,0.032258,0.010507,0.125837,2.0,0.256723,12150,10.37,0,0.18
3,0.024194,0.010841,0.312350,7.0,0.268468,6000,7.88,0,0.09
4,0.080645,0.004837,0.125837,2.0,0.283899,6000,8.49,0,0.18
...,...,...,...,...,...,...,...,...,...
32575,0.048387,0.009540,0.312350,3.0,0.256723,7150,6.99,0,0.12
32576,0.096774,0.009139,0.066606,16.0,0.197457,7000,13.99,0,0.12
32577,0.104839,0.004336,0.312350,2.0,0.146171,6000,19.42,1,0.20
32578,0.153226,0.002385,0.312350,0.0,0.283899,1200,14.74,1,0.07


In [46]:
#df['Payment_History']=df['Payment_History'].apply(lambda value: 0 if value=='N' else 1)

In [47]:
df.corr()


,age,salary,home_ownership,employment_time,loan_purposes,Credit_Amount,loan_rate,Credit_Status,loan_percentage
age,1.000000,0.178899,-0.032980,0.165787,0.053110,0.054172,0.011019,-0.023333,-0.041065
salary,0.178899,1.000000,-0.164789,0.136825,0.000407,0.265879,-0.001381,-0.139938,-0.251511
home_ownership,-0.032980,-0.164789,1.000000,-0.223675,0.052132,-0.104572,0.128023,0.240215,0.107122
employment_time,0.165787,0.136825,-0.223675,1.000000,0.010912,0.110759,-0.056405,-0.082638,-0.055167
loan_purposes,0.053110,0.000407,0.052132,0.010912,1.000000,0.005479,0.013959,0.127439,0.003892
Credit_Amount,0.054172,0.265879,-0.104572,0.110759,0.005479,1.000000,0.145904,0.113550,0.577399
loan_rate,0.011019,-0.001381,0.128023,-0.056405,0.013959,0.145904,1.000000,0.339350,0.123500
Credit_Status,-0.023333,-0.139938,0.240215,-0.082638,0.127439,0.113550,0.339350,1.000000,0.379689
loan_percentage,-0.041065,-0.251511,0.107122,-0.055167,0.003892,0.577399,0.123500,0.379689,1.000000


In [48]:
#transform into some other space
for i in df.select_dtypes(include=np.number):
    if df[i].nunique()!=2:
        print(df['Credit_Status'].corr(df[i]), '--', i)
        print(df['Credit_Status'].corr(df[i].pow(2)), '--', i, '--power')
        print(df['Credit_Status'].corr(np.sqrt(df[i])), '--', i, '--sqrt')
        print(df['Credit_Status'].corr(np.log(df[i])), '--', i, '--log')
        print(df['Credit_Status'].corr(np.sin(df[i])), '--', i, '--sin')
        print(df['Credit_Status'].corr(np.cos(df[i])), '--', i, '--cos')
        print(df['Credit_Status'].corr(1/(1+np.exp(-df[i]))), '--', i, '--sigmoid')

-0.023332610897976842 -- age
-0.01155674961302433 -- age --power
-0.030143462776573992 -- age --sqrt
nan -- age --log
-0.02343758913310915 -- age --sin
0.01168656393160116 -- age --cos
-0.02338857447621656 -- age --sigmoid
-0.13993834495977017 -- salary
-0.01051438186087091 -- salary --power
-0.23289743361150875 -- salary --sqrt
nan -- salary --log
-0.14692883770019313 -- salary --sin
0.011123117383426681 -- salary --cos
-0.14330131399507706 -- salary --sigmoid
0.24021503001066644 -- home_ownership
0.23937694326325792 -- home_ownership --power
0.2396616273590119 -- home_ownership --sqrt
0.23723277138909768 -- home_ownership --log
0.2402139292090958 -- home_ownership --sin
-0.23938762135252534 -- home_ownership --cos
0.24021476472806977 -- home_ownership --sigmoid
-0.08263825431805273 -- employment_time
-0.02150816743581731 -- employment_time --power
-0.09516276833248848 -- employment_time --sqrt
nan -- employment_time --log
0.034624328944584774 -- employment_time --sin
0.02521802311605

In [49]:
#using different trasformation for different columns
df['salary']=np.sqrt(df['salary'])
df['loan_rate']=df['loan_rate'].pow(2)
df['loan_percentage']=np.cos(df['loan_percentage'])

In [50]:
df['Credit_Status'].value_counts()

0    22435
1     6203
Name: Credit_Status, dtype: int64

In [51]:
#create balanced data
ones=df[df['Credit_Status']==1]
zeros=df[df['Credit_Status']==0].sample(frac=0.3)

In [52]:
df=pd.concat([ones, zeros]).sample(frac=1.0)

In [53]:
x=df.loc[:, df.columns!='Credit_Status'].values

y=df['Credit_Status'].values

x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=10)

prediction=RandomForestClassifier()

clf=prediction.fit(x_train, y_train)

result=prediction.predict(x_test)

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, matthews_corrcoef

In [54]:
score1=confusion_matrix(y_test, result)

score2=accuracy_score(y_test, result)

score3=f1_score(y_test, result)

score4=matthews_corrcoef(y_test, result)

In [55]:
score1

array([[1221,  107],
       [ 282,  977]], dtype=int64)

In [56]:
score2

0.8496327792810204

In [57]:
score3

0.8339735381988902

In [58]:
file = open('model.pickle', 'wb')
pickle.dump( clf, file)